Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
size = image_size * image_size

beta = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
        shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
        shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w_1 = tf.Variable(
        tf.truncated_normal([size, size]))
    b_1 = tf.Variable(tf.zeros([size]))
    layer_1 = tf.matmul(tf_train_dataset, w_1) + b_1
    relu_1 = tf.nn.relu(layer_1)
    
    w_2 = tf.Variable(
        tf.truncated_normal([size, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(relu_1, w_2) + b_2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, 
        logits = logits) + 
        beta * tf.nn.l2_loss(w_1) +
        beta * tf.nn.l2_loss(w_2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, w_1) + b_1), w_2) + b_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, w_1) + b_1), w_2) + b_2)

In [6]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
            feed_dict = feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2702.079834
Minibatch accuracy: 14%
Validation accuracy: 30.4%
Minibatch loss at step 500: 16.509079
Minibatch accuracy: 84%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.770987
Minibatch accuracy: 87%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 0.710028
Minibatch accuracy: 83%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.709954
Minibatch accuracy: 84%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 0.699707
Minibatch accuracy: 84%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.853878
Minibatch accuracy: 79%
Validation accuracy: 82.7%
Test accuracy: 89.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (int(step / 1000) * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
            feed_dict = feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2716.024414
Minibatch accuracy: 7%
Validation accuracy: 37.7%
Minibatch loss at step 500: 16.074089
Minibatch accuracy: 100%
Validation accuracy: 71.9%
Minibatch loss at step 1000: 1.270627
Minibatch accuracy: 69%
Validation accuracy: 75.0%
Minibatch loss at step 1500: 0.194639
Minibatch accuracy: 100%
Validation accuracy: 69.8%
Minibatch loss at step 2000: 1.216130
Minibatch accuracy: 69%
Validation accuracy: 70.7%
Minibatch loss at step 2500: 0.198988
Minibatch accuracy: 100%
Validation accuracy: 72.1%
Minibatch loss at step 3000: 1.084490
Minibatch accuracy: 71%
Validation accuracy: 62.9%
Test accuracy: 69.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128
size = image_size * image_size

beta = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
        shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
        shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    drop_1 = tf.nn.dropout(tf_train_dataset, 0.5)
    w_1 = tf.Variable(
        tf.truncated_normal([size, size]))
    b_1 = tf.Variable(tf.zeros([size]))
    layer_1 = tf.matmul(drop_1, w_1) + b_1
    relu_1 = tf.nn.relu(layer_1)
    
    drop_2 = tf.nn.dropout(relu_1, 0.5)
    w_2 = tf.Variable(
        tf.truncated_normal([size, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(drop_2, w_2) + b_2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, 
        logits = logits) + 
        beta * tf.nn.l2_loss(w_1) +
        beta * tf.nn.l2_loss(w_2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.relu(tf.matmul(tf_valid_dataset, w_1) + b_1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction, w_2) + b_2)
    
    test_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, w_1) + b_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction, w_2) + b_2)

In [13]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (int(step / 1000) * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
            feed_dict = feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3042.482422
Minibatch accuracy: 14%
Validation accuracy: 35.6%
Minibatch loss at step 500: 16.408140
Minibatch accuracy: 100%
Validation accuracy: 72.3%
Minibatch loss at step 1000: 1.389917
Minibatch accuracy: 66%
Validation accuracy: 75.1%
Minibatch loss at step 1500: 0.258620
Minibatch accuracy: 100%
Validation accuracy: 69.8%
Minibatch loss at step 2000: 1.210892
Minibatch accuracy: 69%
Validation accuracy: 70.6%
Minibatch loss at step 2500: 0.265186
Minibatch accuracy: 100%
Validation accuracy: 72.1%
Minibatch loss at step 3000: 1.306665
Minibatch accuracy: 64%
Validation accuracy: 67.7%
Test accuracy: 74.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [91]:
batch_size = 128
size = image_size * image_size
beta = 0.0005

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
        shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
        shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w_1 = tf.Variable(
        tf.truncated_normal([size, size], stddev = 0.01))
    b_1 = tf.Variable(tf.zeros([size]))
    layer_1 = tf.matmul(tf_train_dataset, w_1) + b_1
    relu_1 = tf.nn.relu(layer_1)
    
    size_layer_2 = 512
    w_2 = tf.Variable(
        tf.truncated_normal([size, size_layer_2], stddev = 0.01))
    b_2 = tf.Variable(tf.zeros([size_layer_2]))
    layer_2 = tf.matmul(relu_1, w_2) + b_2
    relu_2 = tf.nn.relu(layer_2)
    
    size_layer_3 = 256
    w_3 = tf.Variable(
        tf.truncated_normal([size_layer_2, size_layer_3], stddev = 0.01))
    b_3 = tf.Variable(tf.zeros([size_layer_3]))
    layer_3 = tf.matmul(relu_2, w_3) + b_3
    relu_3 = tf.nn.relu(layer_3)
    
    drop_4 = tf.nn.dropout(relu_3, 0.2)
    w_4 = tf.Variable(
        tf.truncated_normal([size_layer_3, num_labels], stddev = 0.001))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(drop_4, w_4) + b_4
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, 
        logits = logits) + 
        beta * tf.nn.l2_loss(w_1) +
        beta * tf.nn.l2_loss(w_2) +
        beta * tf.nn.l2_loss(w_3) +
        beta * tf.nn.l2_loss(w_4))
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, 
        global_step, decay_steps = 1000, decay_rate = 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, 
        global_step = global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.relu(tf.matmul(tf_valid_dataset, w_1) + b_1)
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, w_2) + b_2)
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, w_3) + b_3)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction, w_4) + b_4)
    
    test_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, w_1) + b_1)
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, w_2) + b_2)
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, w_3) + b_3)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction, w_4) + b_4)

In [92]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
            feed_dict = feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.324758
Minibatch accuracy: 9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.796208
Minibatch accuracy: 78%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.459511
Minibatch accuracy: 88%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 0.535201
Minibatch accuracy: 85%
Validation accuracy: 86.1%
Minibatch loss at step 2000: 0.471013
Minibatch accuracy: 87%
Validation accuracy: 87.2%
Minibatch loss at step 2500: 0.543292
Minibatch accuracy: 87%
Validation accuracy: 87.1%
Minibatch loss at step 3000: 0.700882
Minibatch accuracy: 80%
Validation accuracy: 87.4%
Test accuracy: 93.7%
